In [49]:
import pandas as pd
train_df = pd.read_csv("./ccks_fyh/data/event_entity_train_data_label.csv", sep='\t', header=None, names=['id', 'content', 'type', 'entity'])

## 某个实体出现对应一个类别的概率

In [6]:
train_df.loc[:,'entity'].value_counts()[:15]

权健                641
中信证券              365
权健公司              350
广州证券              315
康美药业              230
安信信托              200
善林金融              145
恒大地产              135
权健集团              128
我爱我家              125
权健自然医学科技发展有限公司    125
陕西坚瑞沃能股份有限公司      117
康得新               117
精功集团              115
中民投               112
Name: entity, dtype: int64

In [13]:
entity_list = train_df.loc[:,'entity'].value_counts()[:15].index

In [17]:
for en in entity_list:
    print(en+':')
    print(train_df[train_df['entity'] == en]['type'].value_counts())

权健:
涉嫌传销      464
涉嫌欺诈      132
涉嫌违法       17
高管负面       15
投诉维权        8
涉嫌非法集资      1
歇业停业        1
失联跑路        1
资产负面        1
不能履职        1
Name: type, dtype: int64
中信证券:
业务资产重组       268
股票转让-股权受让     24
不能履职          24
实际控制人变更       20
交易违规          18
商业信息泄露         6
实控人股东变更        3
业绩下滑           1
财务造假           1
Name: type, dtype: int64
权健公司:
涉嫌传销    182
涉嫌欺诈    111
涉嫌违法     41
高管负面     16
Name: type, dtype: int64
广州证券:
业务资产重组      307
债务违约          7
履行连带担保责任      1
Name: type, dtype: int64
康美药业:
财务信息造假    205
财务造假        8
资金紧张        8
业务资产重组      5
债务违约        2
提现困难        1
涉嫌传销        1
Name: type, dtype: int64
安信信托:
债务违约         195
重组失败           4
股票转让-股权受让      1
Name: type, dtype: int64
善林金融:
涉嫌非法集资    85
涉嫌欺诈      14
涉嫌违法      13
提现困难      10
资产负面       7
高管负面       6
涉嫌传销       5
歇业停业       3
不能履职       2
Name: type, dtype: int64
恒大地产:
业务资产重组       132
股票转让-股权受让      3
Name: type, dtype: int64
权健集团:
涉嫌传销         72
涉嫌欺诈         49
涉嫌违法          3
股票转让-股权受让  

In [2]:
neg_entity = pd.DataFrame(columns=['entity','types','sum','rates'])
entity_list = train_df.loc[:,'entity'].value_counts()[:500].index
for en in entity_list:
    types = train_df[train_df['entity'] == en]['type'].value_counts().index
    value = train_df[train_df['entity'] == en]['type'].value_counts()
    sum = 0;  # 所有类别出现的总数
    for v in range(0,len(value)):
        sum += value[v]
    rate = value[0]/sum
    if (rate > 0.7):  # 最高频次的类别在所有类别中的出现频率 高于70% 则认为是负面实体
        new = pd.DataFrame({'entity': en, 'types': types[0], 'sum': sum,'rates': rate},index=[1])
        neg_entity = neg_entity.append(new,ignore_index=True)
neg_entity = neg_entity.sort_values(by='rates', ascending=False)
neg_entity

,entity,types,sum,rates
163,美餐,业务资产重组,15,1.000000
216,索力鞋业,不能履职,12,1.000000
72,腾邦集团,债务违约,24,1.000000
175,SAC,交易违规,14,1.000000
176,航锦科技,实际控制人变更,14,1.000000
...,...,...,...,...
12,吉利,商业信息泄露,63,0.714286
68,东虹桥担保公司,涉嫌非法集资,24,0.708333
131,置信电气,股票转让-股权受让,17,0.705882
123,金通灵,实际控制人变更,17,0.705882


In [3]:
neg_entity.to_csv('ccks_data_neg_entity.csv',sep='\t')

## neg_entity中实体在dev集中出现

In [78]:
dev_df = pd.read_csv("./ccks_fyh/data/event_entity_dev_data.csv", sep='\t', header=None, names=['id', 'content'])
test_df = pd.read_csv("./ccks_fyh/data/event_entity_data_testB.txt", sep='\t', header=None, names=['id', 'content'])

In [79]:
test_df['len'] = test_df['content'].str.len()
test_df = test_df.drop(columns = ['content'])

In [80]:
test_df.to_csv("testB_content_len.csv", sep='\t',header=0,index=0)

In [54]:
test_df[test_df['content'].str.len() < 16]

,id,content,len
52,600100054,酷派近期的日子的确不太好过,14
129,600100131,我的知识比你们多,8
266,600100268,改革虽难，但方向已定,12
302,600100304,总理伦齐随后辞职,8
315,600100317,行业配置：三条主线精选结构,14
...,...,...,...
267104,600399320,”纳赛尔说,5
267124,600399340,沪深B指延续升势,9
267133,600399349,”杨帆表示,5
267151,600399367,但是这个项目最终也没有做成,13


In [21]:
import numpy as np
train_df = train_df.dropna(axis=0,how='any')

In [33]:
train_df[train_df['content'].str.len() < 14]

,id,content,type,entity
973,2514135,财缘网债务逾期停止发标,提现困难,财缘网
974,2010908,爱投金融提现失败,提现困难,爱投金融
1140,2388497,银湖网为什么提现失败,提现困难,银湖网
1300,2572356,微信提现失败怎么办,提现困难,微信
1309,2553159,金盈所提现失败怎么回事,提现困难,金盈所
...,...,...,...,...
39479,2938458,六国化工：实控人或将变更,实际控制人变更,六国化工
40170,2952617,而乐视网的资金危机，功效,资金紧张,乐视网
40349,2766271,康美药业有预谋实施财务造假,财务信息造假,康美药业
40376,2754149,专家快评ST康美财务造假,财务信息造假,ST康美


## 去空格后首字符为前引号且后面的字符不包括后引号

In [85]:
import re
test_df = pd.read_csv("./ccks_fyh/data/event_entity_data_testB.txt", sep='\t', header=None, names=['id', 'content'])
test_df['content'] = test_df['content'].str.replace(" ", "")
test_df[(test_df['content'].str[0]=='“') & (test_df['content'].str.contains('”')==False)]



,id,content
96,600100098,“这可能会让黄金和白银有机会走高
194,600100196,“导致5月合约惨案的关键因素——储油问题，短期很难改善
219,600100221,“通过简易审理，一起破产清算案件的审理周期可以从3年左右缩短到半年左右
233,600100235,“退市制度是抑制炒作的重要环节，也是健全优胜劣汰的市场机制
245,600100247,“这个策略之下，我们的逻辑跟别的做轻资产公司的不太一样，我们不是纯粹做轻资产的品牌，我们希望...
...,...,...
266841,600399057,“相对来看，理财子公司的净值型理财产品基本符合资管新规要求，信息披露情况相对较好，产品净值波...
266958,600399174,“现在，奔驰、宝马（进口车）的价格都有所上调，奥迪旗下进口车由于不在美生产，目前价格无变化
267030,600399246,“地产板块整体运营效率提升
267034,600399250,“新三板以市场化为导向，必须建立有进有出的市场机制，同时通过不断地提升治理水平，规范挂牌公司...


In [86]:
test_df['quatation'] = ((test_df['content'].str[0]=='“') & (test_df['content'].str.contains('”')==False))
test_df[['id','content','quatation']] = test_df[['id','quatation','content']]

In [87]:
test_df['content'] = test_df['content'].astype("int")
test_df=test_df.drop(columns = ['quatation'])

In [88]:
test_df.to_csv("testB_quotation_rule.csv",sep='\t',header=0,index=0)

In [95]:
train_df = train_df.dropna(axis=0,how='any')
train_df['content'] = train_df['content'].str.replace(" ", "")
train_df[(train_df['content'].str[0]=='“') & (train_df['content'].str.contains('”')==False)].describe()

,id
count,4.900000e+01
mean,2.556649e+06
std,3.064524e+05
min,2.037104e+06
25%,2.255676e+06
50%,2.575273e+06
75%,2.793035e+06
max,3.075910e+06


In [109]:
49/41000

0.0011951219512195122

In [110]:
5814/267000

0.021775280898876405

# 后引号开头

In [111]:
test_df = pd.read_csv("./ccks_fyh/data/event_entity_data_testB.txt", sep='\t', header=None, names=['id', 'content'])
test_df['content'] = test_df['content'].str.replace(" ", "")
test_df[(test_df['content'].str[0]=='”')]

,id,content
21,600100023,”刘晖告诉记者，未来，新华都集团可能会借云南白药的品牌影响力或者供应商、经销商资源做一些“自...
59,600100061,”如此“扎眼”的工资差距是否合理呢？工资是市场、企业、劳动者等各市场主体多次博弈的结果，是在...
88,600100090,”数据显示，短期债务中公司账面贸易应付款及应付票据自2016年的19.87亿增加至2018年...
107,600100109,”这是他对该议题最强硬的评论之一
120,600100122,”该销售人员表示，目前在售的房源是北区17号楼，20多平方米的小户型已经售罄，38平方米和5...
...,...,...
267104,600399320,”纳赛尔说
267126,600399342,”更尴尬的是，这位员工还被套了
267133,600399349,”杨帆表示
267135,600399351,”然而在中国城市产业发展联盟主席陈宝存看来，北京作为首都，作为全国监管最为严格的城市，北京市...


In [112]:
test_df['quotation'] = (test_df['content'].str[0]=='”')
test_df[['id','content','quotation']] = test_df[['id','quotation','content']]
test_df['content'] = test_df['content'].astype("int")
test_df=test_df.drop(columns = ['quotation'])
test_df.to_csv("testB_quotation_rule2.csv",sep='\t',header=0,index=0)

In [96]:
train_df['content'] = train_df['content'].str.replace(" ", "")
train_df[(train_df['content'].str[0]=='”')]

,id,content,type,entity
846,2714638,”与此同时，微商贷还在展期公告里公布了详细的兑付计划，，称自8月5日至10月31日，所有本金...,提现困难,微商贷
964,2584354,”6月1日，健康猫系统升级调试公告但在冲刺的另一面，健康猫从5月起再次出现无法提现的情况,提现困难,健康猫
1049,2641259,” 四 就在今年5月无法提现的危机出现之后，大象科技在6月宣布完成C轮5亿元的融资,提现困难,大象科技
1142,2760561,”四就在今年5月无法提现的危机出现之后，大象科技在6月宣布完成C轮5亿元的融资,提现困难,大象科技
1439,2449823,” 宜湃网逾期遭实控人甩锅，宜贷网股东会决议对其报警立案 此外，在宜贷网“良性退出”公告...,提现困难,宜湃网
...,...,...,...,...
40010,2724147,”2019年1月31日，雏鹰农牧则公开表示：“2018年6月开始，公司出现资金流动性紧张局面...,资金紧张,雏鹰农牧
40266,2736464,”乐视网信息技术(北京)股份有限公司在2017年三季报中称，由于受到关联方资金紧张风波影响，...,资金紧张,乐视网信息技术(北京)股份有限公司
40286,2888612,”据业内人士称，周克为人极具个人魅力，“虽然一直非常坚强地与病魔抗争，但他工作勤奋，非常敬业...,资金紧张,中邮创业基金管理股份有限公司
40343,2208516,”储继荣告诉记者，随着生产规模的扩大，所需资金量也在增加，每天都要10万元左右的资金流才能确...,资金紧张,富滇银行


In [97]:
237/41000

0.005780487804878049

In [108]:
11659/267000

0.043666666666666666

## 训练集NaN数据中的规则分布

In [99]:
train_df = pd.read_csv("./ccks_fyh/data/event_entity_train_data_label.csv", sep='\t', header=None, names=['id', 'content', 'type', 'entity'])

In [100]:
train_nan = train_df[train_df['type'].isna()]
train_nan

,id,content,type,entity
41067,2431217,（3）利润来源情况 2017年度、2018年1-6月，穗甬控股主要利润来源情况如下： 单位：...,NaN,NaN
41068,2244962,同时，责成大丰区委、区政府作出深刻检查；对江苏辉丰生物农业股份有限公司存在环境问题的11条生...,NaN,NaN
41069,2372908,【八项承诺】在成都国际车展上，东风雪铁龙提出了八项无忧承诺：常规保养2小时内完成，逾期不用付...,NaN,NaN
41070,2829316,霸气微信群名众筹与免费的确为大可乐手机带来了关注度，不过，这种“新颖”的，但稍显不成熟的营销...,NaN,NaN
41071,2507518,除了广告业务之外，要知道，今日头条推出股票资讯平台“钠镁股票”，不同于同花顺等软件，今日头条...,NaN,NaN
...,...,...,...,...
66756,2568107,2019年以来，包括牛津大学、麻省理工学院等欧美顶尖学校，由于担心失去政府的资金支持，纷纷宣...,NaN,NaN
66757,2688414,"渣打无抵押贷款,拍拍贷网：美国知名招聘网站Glassdoor日前公布了2016年度全美薪酬的...",NaN,NaN
66758,2415192,如果想要买三维植被网，就选择晋中三维植被网厂家报价--欢迎您记者7月21日从保定天威保变电气...,NaN,NaN
66759,2983848,同时能较好地维养护工业凹隐私数据，投降低工业边际终端敏感数据凹隐私泄露的风险中壹签到微少赚2...,NaN,NaN


In [103]:
train_nan['content'] = train_nan['content'].str.replace(" ", "")
train_nan[(train_nan['content'].str[0]=='“')& (train_df['content'].str.contains('”')==False)].describe()

/Users/lixiang/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/lixiang/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


,id
count,3.000000e+01
mean,2.609246e+06
std,3.121694e+05
min,2.127852e+06
25%,2.405613e+06
50%,2.559746e+06
75%,2.920115e+06
max,3.105094e+06


In [104]:
30/25694

0.0011675877636802365

In [106]:
train_nan['content'] = train_nan['content'].str.replace(" ", "")
train_nan[(train_nan['content'].str[0]=='”')].describe()

/Users/lixiang/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,id
count,7.000000e+01
mean,2.628707e+06
std,2.972017e+05
min,2.110957e+06
25%,2.416134e+06
50%,2.589868e+06
75%,2.898403e+06
max,3.123531e+06


In [107]:
70/25694

0.0027243714485872188

In [105]:
train_nan[train_nan['content'].str.len() < 14]

,id,content,type,entity
41184,2120014,捷信逾期欠款怎么办还不上,NaN,NaN
41292,2502238,华霖资本被骗亏损能挽回吗,NaN,NaN
41328,2791226,协助悠融对逾期项目进行催收,NaN,NaN
41357,2184662,处于博莱，泰莱,NaN,NaN
41370,2794014,小米贷款延期,NaN,NaN
...,...,...,...,...
66066,3142324,工行逾期罚金如何计算的？,NaN,NaN
66307,2883638,ST银亿7月22日打开跌停,NaN,NaN
66387,2810327,*ST菲达,NaN,NaN
66526,2611882,关于建行逾期自动扣款问题！,NaN,NaN


In [5]:
over20 = neg_entity[neg_entity['sum']>=20]
golden_list = over20[over20['rates']==1.00]
golden_list

,entity,types,sum,rates
72,腾邦集团,债务违约,24,1.0
44,英大证券,业务资产重组,31,1.0
96,武钢集团,业务资产重组,20,1.0
90,浙江吉利汽车研究院有限公司,商业信息泄露,21,1.0
85,网信证券,债务违约,22,1.0
83,中天金融集团股份有限公司,业务资产重组,22,1.0
80,广州越秀金融控股集团股份有限公司,业务资产重组,22,1.0
67,湘财证券,业务资产重组,24,1.0
75,北京信威科技集团股份有限公司,业务资产重组,23,1.0
18,广州证券股份有限公司,业务资产重组,57,1.0


In [6]:
golden_entity = list(golden_list['entity'])
golden_type = list(golden_list['types'])

In [72]:
submit_df = pd.read_csv("./ccks_fyh/data/rule_no_1.2.csv", sep='\t', header=None, names=['id', 'type', 'entity']) 

In [8]:
test_df.head()

,id,content
0,600100002,西藏旅游对第一次问询函的答复显示，公司于2016年1月底将4亿元归还银行
1,600100003,接下来网商银行APP也会陆续接入资金增值、信用服务、风险管理等更加丰富的综合金融服务
2,600100004,今年以来白马股表现十分抢眼，尤其是家电和白酒板块
3,600100005,中国银行业协会专职副会长杨再平也表示，银行业应该确定适度利润目标，中国银行业的盈利能力仍然处...
4,600100006,而受到债市走熊拖累，多只债券基金过去一年净值大跌


In [11]:
hard_entity_in_dev = pd.DataFrame(columns=['id','content'])
count = 0;
for en in golden_entity:
    find=test_df[test_df['content'].str.contains(en)]
    if find.empty==False:
        count += 1
        # hard_entity_in_dev.append(find)
        print(find)
#     find['entity']=en
#     hard_entity_in_dev.append(find)
hard_entity_in_dev.head()
print(count)

               id                                            content
48084   600148086                不过，两个月后，腾邦集团和钟百胜却单方面解除协议，双方还为此事闹上法院
71181   600171183                  腾邦国际:腾邦集团累计被动减持5.78万#MEY5##MEY5#股
111903  600211905  最为市场关注的一次，便是2019年6月，腾邦集团和实控人钟百胜拟将腾邦国际28.30%的股份...
118685  600218687   数据显示，腾邦集团及钟百胜共持有公司股份1.71亿股，占公司总股本的27.80%；其所持有...
136696  600236698                        腾邦国际:腾邦集团累计被动减持5.78万#MEY5#股
222818  600355034  当日，腾邦国际控股股东腾邦集团公告称，因短期资金周转困难，“17腾邦01”债券未能按时足额支...
               id                                            content
4461    600104463              公司表示，公司债券的偿债资金将主要来源于英大证券日常经营所产生的收入及利润
4965    600104967  公司在全国范围内设有35家分支机构，并控股英大期货有限公司、英大证券投资有限公司，拥有包括证...
5020    600105022   英大证券研究所所长郑后成告诉第一财经，国内循环是推动我国宏观经济发展的内部驱动力，而国际循...
8236    600108238           英大证券经营稳健，信用记录良好，各项风险监管指标均满足相关要求，外部融资渠道畅通
8428    600108430   上半年国网英大完成资产重组，净利润超2019年全年 2020年上半年，国网英大完成重大资产...
35943   600135945    （自营收入=投资收益+公允价值变动收益-对联营企业和合营企业的投资收益，按2020年半年...
58782   600158784                而

In [67]:
dev_df.head()

,id,content
0,2527248,腾讯证券5月23日讯，*ST康得收到关注函：公司控股股东康得投资集团有限公司信托贷款出现违约...
1,2182665,2017年2月15日，绍兴市越城区人民法院作出民事裁判书，其中作为连带担保方的浙江一景乳业股...
2,2601297,4、被告王青峰、陈灵金、广东创富融资担保有限公司对被告广州吉福物流配送有限公司的前述各项总款...
3,2623216,判决为：限杨丽、曹浩于生效之日起10日内偿付原告108277.33元，自2017年1月19日...
4,2401535,去年10月，桂林中院判决阳朔一尺水实业投资开发有限公司、广西印象刘三姐旅游文化产业投资有限责...


## 某个实体出现对应两个类别

In [82]:

double_df = pd.DataFrame(columns=['entity','type'])
for i, line in train_df.groupby(['content', 'entity']):
    
    if len(line)!=1:
        for en in golden_entity:
            if line['entity']==en:
                
        print(line)

            id                                            content       type  \
33841  2362354         中兴商业（ 000715 ）公告，辽宁方大集团拟受让公司控股股东所持29%股份...  股票转让-股权受让   
34047  2408504         中兴商业（ 000715 ）公告，辽宁方大集团拟受让公司控股股东所持29%股份...  股票转让-股权受让   

       entity  
33841  沈阳盛京资产  
34047  沈阳盛京资产  
            id                                            content       type  \
33842  2911310         中兴商业（ 000715 ）公告，辽宁方大集团拟受让公司控股股东所持29%股份...  股票转让-股权受让   
34048  2289018         中兴商业（ 000715 ）公告，辽宁方大集团拟受让公司控股股东所持29%股份...  股票转让-股权受让   

       entity  
33842  辽宁方大集团  
34048  辽宁方大集团  
            id                                            content    type  \
4592   2657234   #吉林文投##吉林文投集团#【长春公安：以涉嫌非法吸收公众存款罪对吉林文投集团法人张子毅等...    涉嫌违法   
14124  2476893   #吉林文投##吉林文投集团#【长春公安：以涉嫌非法吸收公众存款罪对吉林文投集团法人张子毅等...  涉嫌非法集资   

       entity  
4592   吉林文投集团  
14124  吉林文投集团  
            id                                            content    type  \
13956  2772720   #揭露健康猫非法集资[超话]##健康猫诈骗受害者##健康猫诈骗##何炅[超话]#健康猫诈骗...

14478   萨飞资本  
            id                                          content    type entity
13063  2062830   2018年7月，今金贷陆续被传出现逾期，同时存在与其他平台标的互换、多头借贷、自融等违规行为  涉嫌非法集资    今金贷
16018  2287580   2018年7月，今金贷陆续被传出现逾期，同时存在与其他平台标的互换、多头借贷、自融等违规行为    资产负面    今金贷
            id                                            content  type entity
7029   2486346   2018年9月11日，该公司网站发布公告称，假借与天狮公司合作、对外宣称是“天津天狮”或“...  涉嫌传销   天津天狮
19135  2120197   2018年9月11日，该公司网站发布公告称，假借与天狮公司合作、对外宣称是“天津天狮”或“...  涉嫌欺诈   天津天狮
            id                                            content  type entity
7453   2313007   2018年9月11日，该公司网站发布公告称，假借与天狮公司合作、对外宣称是“天津天狮”或“...  涉嫌传销   天狮公司
19959  2124856   2018年9月11日，该公司网站发布公告称，假借与天狮公司合作、对外宣称是“天津天狮”或“...  涉嫌欺诈   天狮公司
            id                                            content  type  \
7924   2512016   2018年9月11日，该公司网站发布公告称，假借与天狮公司合作、对外宣称是“天津天狮”或“...  涉嫌传销   
19124  2637096   2018年9月11日，该公司网站发布公告称，假借与天狮公司合作、对外宣称是“天津天狮”或“...  涉嫌欺诈   

       entity  
7924   天狮生物发展  
19124  天狮生物发展  


13613  宜信卓越财富投资管理（北京）有限公司  
            id                                            content    type  \
13611  2319240   ?2015年7月24日，大连****发布《涉嫌非法集资广告的通告》，36家P2P平台被通报...  涉嫌非法集资   
14511  2789495   ?2015年7月24日，大连****发布《涉嫌非法集资广告的通告》，36家P2P平台被通报...  涉嫌非法集资   

      entity  
13611   恒昌财富  
14511   恒昌财富  
            id                                            content    type  \
13615  2841109   ?2015年7月24日，大连****发布《涉嫌非法集资广告的通告》，36家P2P平台被通报...  涉嫌非法集资   
13998  2306504   ?2015年7月24日，大连****发布《涉嫌非法集资广告的通告》，36家P2P平台被通报...  涉嫌非法集资   

             entity  
13615  投资管理（北京）有限公司  
13998  投资管理（北京）有限公司  
            id                                            content    type  \
13612  2392280   ?2015年7月24日，大连****发布《涉嫌非法集资广告的通告》，36家P2P平台被通报...  涉嫌非法集资   
14127  2069094   ?2015年7月24日，大连****发布《涉嫌非法集资广告的通告》，36家P2P平台被通报...  涉嫌非法集资   

           entity  
13612  放心（北京）有限公司  
14127  放心（北京）有限公司  
            id                                            content    type  \
4511   2882942   ???2018年7月1

11557  小诸葛金服  
            id                                            content    type  \
4602   2578389   [[+_+]]暴雷潮不断上演“监管救市”戏码6月15日，知名P2P平台唐小僧发布系统升级公...    涉嫌违法   
5321   2712970   [[+_+]]暴雷潮不断上演“监管救市”戏码6月15日，知名P2P平台唐小僧发布系统升级公...    涉嫌违法   
14728  2625380   [[+_+]]暴雷潮不断上演“监管救市”戏码6月15日，知名P2P平台唐小僧发布系统升级公...  涉嫌非法集资   

      entity  
4602     投融家  
5321     投融家  
14728    投融家  
            id                                            content    type  \
3881   2101272   [[+_+]]暴雷潮不断上演“监管救市”戏码6月15日，知名P2P平台唐小僧发布系统升级公...    失联跑路   
4603   2915769   [[+_+]]暴雷潮不断上演“监管救市”戏码6月15日，知名P2P平台唐小僧发布系统升级公...    涉嫌违法   
5322   2211193   [[+_+]]暴雷潮不断上演“监管救市”戏码6月15日，知名P2P平台唐小僧发布系统升级公...    涉嫌违法   
10415  2886318   [[+_+]]暴雷潮不断上演“监管救市”戏码6月15日，知名P2P平台唐小僧发布系统升级公...    投诉维权   
13661  2683922   [[+_+]]暴雷潮不断上演“监管救市”戏码6月15日，知名P2P平台唐小僧发布系统升级公...  涉嫌非法集资   

       entity  
3881   永利宝APP  
4603   永利宝APP  
5322   永利宝APP  
10415  永利宝APP  
13661  永利宝APP  
            id                                           

11423    永利宝  
            id                                            content  type  \
4202   2819865   　　2019年1月1日，天津市公安机关对权健自然医学科技发展有限公司涉嫌组织、领导传销活动...  涉嫌违法   
4312   2377229   　　2019年1月1日，天津市公安机关对权健自然医学科技发展有限公司涉嫌组织、领导传销活动...  涉嫌违法   
7782   2109887   　　2019年1月1日，天津市公安机关对权健自然医学科技发展有限公司涉嫌组织、领导传销活动...  涉嫌传销   
8785   2088628   　　2019年1月1日，天津市公安机关对权健自然医学科技发展有限公司涉嫌组织、领导传销活动...  涉嫌传销   
18860  2954759   　　2019年1月1日，天津市公安机关对权健自然医学科技发展有限公司涉嫌组织、领导传销活动...  涉嫌欺诈   
19267  2909239   　　2019年1月1日，天津市公安机关对权健自然医学科技发展有限公司涉嫌组织、领导传销活动...  涉嫌欺诈   

               entity  
4202   权健自然医学科技发展有限公司  
4312   权健自然医学科技发展有限公司  
7782   权健自然医学科技发展有限公司  
8785   权健自然医学科技发展有限公司  
18860  权健自然医学科技发展有限公司  
19267  权健自然医学科技发展有限公司  
            id                                            content    type  \
4200   2193164   　　21世纪经济报道记者多方获悉，警方已对此立案调查，长来财富法定代表人已被警方控制，部分...    涉嫌违法   
14664  2194947   　　21世纪经济报道记者多方获悉，警方已对此立案调查，长来财富法定代表人已被警方控制，部分...  涉嫌非法集资   

      entity  
4200    长来财富  
14664   长来财富  
            id

20197  钰诚国际控股集团  
            id                                            content    type  \
1265   2130925   【直报网北京12月10日讯】(现代快报)12月6日，南京建邺警方通报，今年5月初，“爱晚系...    提现困难   
3696   2973870   【直报网北京12月10日讯】(现代快报)12月6日，南京建邺警方通报，今年5月初，“爱晚系...    失联跑路   
13849  2520423   【直报网北京12月10日讯】(现代快报)12月6日，南京建邺警方通报，今年5月初，“爱晚系...  涉嫌非法集资   

      entity  
1265     爱晚系  
3696     爱晚系  
13849    爱晚系  
            id                                            content    type  \
15372  2815942   【直报网北京12月29日讯】(庞氏骗局快讯)近日，有网友联系庞氏骗局快讯咨询，通过一家叫“...  涉嫌非法集资   
19580  2177593   【直报网北京12月29日讯】(庞氏骗局快讯)近日，有网友联系庞氏骗局快讯咨询，通过一家叫“...    涉嫌欺诈   

      entity  
15372   理想家园  
19580   理想家园  
            id                                            content    type  \
11924  2531966   【突发】信和财富、金瑞龙财富、冠群等机构被??日前，天津市市场和质量监督管?18砦?被嵊》...  涉嫌非法集资   
11946  2831191   【突发】信和财富、金瑞龙财富、冠群等机构被??日前，天津市市场和质量监督管?18砦?被嵊》...  涉嫌非法集资   

      entity  
11924   信和财富  
11946   信和财富  
            id                                            c

14487  浙江草根网络科技有限公司  
            id                                            content    type  \
12574  2834417   全体高通汽车人顿时蒙圈，并陷入一阵哭泣声中!4月24日晚间，上海官方消息，高通汽车老板，也...  涉嫌非法集资   
18475  2456208   全体高通汽车人顿时蒙圈，并陷入一阵哭泣声中!4月24日晚间，上海官方消息，高通汽车老板，也...    高管负面   

      entity  
12574   善林金融  
18475   善林金融  
            id                                            content  type  \
8595   2750985   公司三名骨干被批捕另据南山警方此前的通报——9月4日深圳市牛牛通宝科技文化产业有限公司（“...  涉嫌传销   
18461  2183771   公司三名骨干被批捕另据南山警方此前的通报——9月4日深圳市牛牛通宝科技文化产业有限公司（“...  高管负面   

                  entity  
8595   深圳市牛牛通宝科技文化产业有限公司  
18461  深圳市牛牛通宝科技文化产业有限公司  
            id                                            content    type  \
4078   2132521   公安机关对投资人给予的协助配合表示衷心感谢！智佳金服登记详细公告合肥市公安局庐阳分局正在侦...    涉嫌违法   
12034  2271088   公安机关对投资人给予的协助配合表示衷心感谢！智佳金服登记详细公告合肥市公安局庐阳分局正在侦...  涉嫌非法集资   

          entity  
4078   大志投资集团有限公  
12034  大志投资集团有限公  
            id                                            content    type  \
5060   2037718   公安机关对投资人给予的

18554  安徽星烁投资管理有限公司  
            id                                            content    type  \
5049   2657897   在本案中，被告宏旺公司相关人员因涉嫌非法集资被夏津县公安机关立案侦查，因此本案已经不属于单...    涉嫌违法   
14828  2627110   在本案中，被告宏旺公司相关人员因涉嫌非法集资被夏津县公安机关立案侦查，因此本案已经不属于单...  涉嫌非法集资   

      entity  
5049    宏旺公司  
14828   宏旺公司  
            id                                            content  type entity
4590   2428433   在短短不到半个月的时间里，曾经盛极一时的权健“直销帝国”就成为了人人喊打的过街老鼠，在民意...  涉嫌违法     权健
9006   2160994   在短短不到半个月的时间里，曾经盛极一时的权健“直销帝国”就成为了人人喊打的过街老鼠，在民意...  涉嫌传销     权健
19417  2615652   在短短不到半个月的时间里，曾经盛极一时的权健“直销帝国”就成为了人人喊打的过街老鼠，在民意...  涉嫌欺诈     权健
            id                                      content    type entity
4995   2430249   在荣海鑫源多名现金客户报案之后，公司法人代表张某被顺义警方以非法吸收公众存款罪立案了    涉嫌违法   荣海鑫源
14172  2691387   在荣海鑫源多名现金客户报案之后，公司法人代表张某被顺义警方以非法吸收公众存款罪立案了  涉嫌非法集资   荣海鑫源
           id                                            content  type  \
4406  2938073   基本案情：2018年8月28日，佛山市南海区市场监管局对佛山市皓铭化妆品有限公司涉嫌以传销...  涉嫌违法   
7877  220099

19835  2930838   文章引发关注后，权健随即不断被起底，包括以保健品形式兜售的各类“产品”，动辄称之为“抗癌秘...  涉嫌欺诈     权健
            id                           content  type entity
7805   2460641   新京报记者注意到，此前有关无限极虚假宣传、涉嫌传销的质疑声不断  涉嫌传销    无限极
19101  2284198   新京报记者注意到，此前有关无限极虚假宣传、涉嫌传销的质疑声不断  涉嫌欺诈    无限极
            id                                            content    type  \
14284  2880745   新华社南京７月２７日消息，记者从南京市公安局了解到，２７日，南京市公安局江北新区分局依法对...  涉嫌非法集资   
18654  2230511   新华社南京７月２７日消息，记者从南京市公安局了解到，２７日，南京市公安局江北新区分局依法对...    高管负面   

      entity  
14284    钱宝系  
18654    钱宝系  
            id                                            content  type entity
8024   2223131   新华社石家庄1月16日电（记者杨帆）近日有媒体曝光了河北华林酸碱平生物技术有限公司（以下简...  涉嫌传销   华林公司
18886  2060333   新华社石家庄1月16日电（记者杨帆）近日有媒体曝光了河北华林酸碱平生物技术有限公司（以下简...  涉嫌欺诈   华林公司
            id                                            content  type  \
10176  2143372   新华社石家庄1月16日电（记者杨帆）近日有媒体曝光了河北华林酸碱平生物技术有限公司（以下简...  涉嫌传销   
18957  2823935   新华社石家庄1月16日电（记者杨帆）近日有媒体曝光了河北华林酸碱平生物技术有限公司（以下简...  涉嫌欺诈

KeyboardInterrupt: 

In [84]:
with open('test.txt', 'a') as f:
    f.write('hello\n')
    f.write('world')